In [ ]:
import pandas as pd
import numpy as np
from sklearn import metrics

In [ ]:
df = pd.read_csv('data_for_predictions.csv')

In [ ]:
df.head(6)

,Unnamed: 0,id,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,...,months_modif_prod,months_renewal,channel_MISSING,channel_ewpakwlliwisiwduibdlfmalxowmwpci,channel_foosdfpfkusacimwkcsosbicdxkicaua,channel_lmkebamcaaclubfxadlmueccxoimlema,channel_usilxuppasemubllopkaafesmlibmsdf,origin_up_kamkkxfxxuwbdslkwifmmcsiusiuosws,origin_up_ldkssxwpmemidmecebumciepifcamkci,origin_up_lxidpiddsbxsbosboudacockeimpuepw
0,0,24011ae4ebbe3035111d65fa7c15bc57,0.000000,4.739944,0.000000,0.000000,0.0,0.444045,0.114481,0.098142,...,2.0,6.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,1,d29c2c54acc38ff3c0614d0a653813dd,3.668479,0.000000,0.000000,2.280920,0.0,1.237292,0.145711,0.000000,...,76.0,4.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2,764c75f661154dac3a6c254cd082ea7d,2.736397,0.000000,0.000000,1.689841,0.0,1.599009,0.165794,0.087899,...,68.0,8.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,3,bba03439a292a1e166f80264c16191cb,3.200029,0.000000,0.000000,2.382089,0.0,1.318689,0.146694,0.000000,...,69.0,9.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,4,149d57cf92fc41cf94415803a877cb4b,3.646011,0.000000,2.721811,2.650065,0.0,2.122969,0.116900,0.100015,...,71.0,9.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,5,1aa498825382410b098937d65c4ec26d,3.919235,0.000000,3.300813,2.901970,0.0,1.493040,0.164775,0.086131,...,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


#remove unnamed from the dataset

In [ ]:
# Check if the column exists before dropping
if "Unnamed: 0" in df.columns:
    df.drop(columns=["Unnamed: 0"], inplace=True)
else:
    print("Column 'Unnamed: 0' not found in DataFrame.")

#Spliting dataset into training set and test set

In [ ]:
y = df['churn']
X = df.drop(columns=['id', 'churn'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(2417, 61)
(2417,)
(806, 61)
(806,)


In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier


In [ ]:
# Create an imputer to replace missing values (NaN) with the mean of each column
imputer = SimpleImputer(strategy='mean')

In [ ]:
# Fit the imputer on the training data and transform both training and testing data
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

In [ ]:
# Impute missing values in the target variable (y_train)
imputer_y = SimpleImputer(strategy='most_frequent')  # Use most frequent for categorical target
y_train = imputer_y.fit_transform(y_train.values.reshape(-1, 1))  # Reshape for single feature
y_train = y_train.ravel() # Flatten the array

#training our dataset using Random forest Classifier

In [ ]:
model = RandomForestClassifier(n_estimators=1000)
model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=1000)

#finding R_2 score for random forest

In [ ]:
from sklearn.metrics import r2_score

y_train_pred = model.predict(X_train) # Predict on the training data
r2_score_train = r2_score(y_train, y_train_pred) # Compare true values with predictions
print("R2 score on training data:", r2_score_train)

R2 score on training data: 1.0


Since the R_2 score for random forest was the highest thats why we choose this model

#Evalution

Now let's evaluate how well this trained model is able to predict the values of the test dataset.

We are going to use 3 metrics to evaluate performance:

- Accuracy = the ratio of correctly predicted observations to the total observations
- Precision = the ability of the classifier to not label a negative sample as positive
- Recall = the ability of the classifier to find all the positive samples

In [ ]:
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix,accuracy_score, precision_score, recall_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)
precision = precision_score(y_test, y_pred)
print(precision)
recall = recall_score(y_test, y_pred)
print(recall)


[[721   1]
 [ 82   2]]
0.8970223325062034
0.6666666666666666
0.023809523809523808
